In [ ]:
!pip install datasets
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

In [ ]:
dataset_news = load_dataset("ag_news")
count0, count1, count2, count3 = 0, 0, 0, 0
dataset_short = []
for i in range(len(dataset_news['train'])):
  if dataset_news['train'][i]['label'] == 0 and count0 < 3000:
    dataset_short.append({'news': dataset_news['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_news['train'][i]['label'] == 1 and count1 < 3000:
    dataset_short.append({'news': dataset_news['train'][i]['text'], 'label': 1})
    count1 += 1
  elif dataset_news['train'][i]['label'] == 2 and count2 < 3000:
    dataset_short.append({'news': dataset_news['train'][i]['text'], 'label': 2})
    count2 += 1
  elif dataset_news['train'][i]['label'] == 3 and count3 < 3000:
    dataset_short.append({'news': dataset_news['train'][i]['text'], 'label': 3})
    count3 += 1
len(dataset_short)

In [ ]:
import random
random.shuffle(dataset_short)
dataset_news = {'train': dataset_short[:9600], 'test': dataset_short[9600:]}

In [ ]:
# Выводим объём выборок
print('Объём train:', len(dataset_news['train']))
print('Объём test:', len(dataset_news['test']))

train_texts_news = [s['news'] for s in dataset_news['train']]
test_texts_news = [s['news'] for s in dataset_news['test']]

train_labels_news = [s['label'] for s in dataset_news['train']]
test_labels_news = [s['label'] for s in dataset_news['test']]

# ВЫводим примеры
print(train_texts_news[0], train_labels_news[0])
print(test_texts_news[0], test_labels_news[0])

In [ ]:
# второй датасет
dataset_imdb = load_dataset("imdb")

count0, count1 = 0, 0
dataset_short_imdb = []

for i in range(len(dataset_imdb['train'])):
  if dataset_imdb['train'][i]['label'] == 0 and count0 < 6000:
    dataset_short_imdb.append({'text': dataset_imdb['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_imdb['train'][i]['label'] == 1 and count1 < 6000:
    dataset_short_imdb.append({'text': dataset_imdb['train'][i]['text'], 'label': 1})
    count1 += 1
len(dataset_short_imdb)

In [ ]:
random.shuffle(dataset_short_imdb)
dataset_imdb = {'train': dataset_short_imdb[:9600], 'test': dataset_short_imdb[9600:]}

In [ ]:
# Выводим объём выборок
print('Объём train:', len(dataset_imdb['train']))
print('Объём test:', len(dataset_imdb['test']))

train_texts_imdb = [s['text'] for s in dataset_imdb['train']]
test_texts_imdb = [s['text'] for s in dataset_imdb['test']]

train_labels_imdb = [s['label'] for s in dataset_imdb['train']]
test_labels_imdb = [s['label'] for s in dataset_imdb['test']]

print(train_texts_imdb[0])
print(train_labels_imdb[0])
print()
print(test_texts_imdb[0])
print(test_labels_imdb[0])

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.metrics import f1_score
stop_words = list(set(stopwords.words("english")))

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
def base_preprocessing(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Zа-яА-Я ]', '', text)
    tokens = word_tokenize(text)
    return tokens

In [ ]:
def stopwords_special(text):
  tokens = base_preprocessing(text)
  tokens = [word for word in tokens if word not in stop_words]  # убираем стоп-слова
  return tokens

In [ ]:
def stemming(text):
  tokens = stopwords_special(text)

  stemmer = nltk.PorterStemmer()  # инициализируем стеммер
  stemmed_tokens = [stemmer.stem(token) for token in tokens]  # перебираем токены и применяем алгоритм стемминга
  return stemmed_tokens

In [ ]:
def lemmatisation(text):
  tokens = stopwords_special(text)

  lemma = nltk.WordNetLemmatizer()
  lemma_tokens = [lemma.lemmatize(token) for token in tokens]
  return lemma_tokens

In [ ]:
def vectorize(train_texts, test_texts, method, edit):
  if method == "count":
    vectorizer = CountVectorizer()
  else:
    vectorizer = TfidfVectorizer()

  # edit = 1 (stopwords_specoal), 2 (stemming), 3 (lemmatisation)
  if edit == 1:
    train_texts = [' '.join(stopwords_special(text)) for text in train_texts]
    test_texts = [' '.join(stopwords_special(text)) for text in test_texts]
  elif edit == 2:
    train_texts = [' '.join(stemming(text)) for text in train_texts]
    test_texts = [' '.join(stemming(text)) for text in test_texts]
  elif edit == 3:
    train_texts = [' '.join(lemmatisation(text)) for text in train_texts]
    test_texts = [' '.join(lemmatisation(text)) for text in test_texts]

  train_text = vectorizer.fit_transform(train_texts)
  test_text = vectorizer.transform(test_texts)

  return train_text, test_text

In [ ]:
results = []
for sizes in [(20, 40,), (20, 20, 20,), (10, 10, 20, 20)]:
  for d in ['news', 'imdb']:
    for m in ['count', 'tfidf']:
      for edit in [1, 2, 3]:
        if d == 'news':
          train_texts = train_texts_news
          test_texts = test_texts_news
          train_labels = train_labels_news
          test_labels = test_labels_news
        else:
          train_texts = train_texts_imdb
          test_texts = test_texts_imdb
          train_labels = train_labels_imdb
          test_labels = test_labels_imdb


        train_text, test_text = vectorize(train_texts, test_texts, m, edit)

        mlp_clf = MLPClassifier(hidden_layer_sizes=sizes)
        mlp_clf.fit(train_text, train_labels)
        mlp_pred = mlp_clf.predict(test_text)
        if edit == 1:
          results.append([sizes, d, m, 'stopwords_special', f1_score(test_labels, mlp_pred, average='macro')])
        elif edit == 2:
          results.append([sizes, d, m, 'stemming', f1_score(test_labels, mlp_pred, average='macro')])
        elif edit == 3:
          results.append([sizes, d, m, 'lemmatisation', f1_score(test_labels, mlp_pred, average='macro')])

        if len(results) % 10 == 0:
          print(len(results))

In [ ]:
len(results)

In [ ]:
df_results = pd.DataFrame(results, columns=["hidden_layer_sizes", "Dataset", "Vectorizer", "Preprocess", "F1-score"])
df_results.to_excel("results_mlp.xlsx", index=False)

In [ ]:
results = []
for sizes in [(20, 20, 30, 50)]:
  for d in ['news', 'imdb']:
    for m in ['count', 'tfidf']:
      for edit in [1, 2, 3]:
        if d == 'news':
          train_texts = train_texts_news
          test_texts = test_texts_news
          train_labels = train_labels_news
          test_labels = test_labels_news
        else:
          train_texts = train_texts_imdb
          test_texts = test_texts_imdb
          train_labels = train_labels_imdb
          test_labels = test_labels_imdb


        train_text, test_text = vectorize(train_texts, test_texts, m, edit)

        mlp_clf = MLPClassifier(hidden_layer_sizes=sizes)
        mlp_clf.fit(train_text, train_labels)
        mlp_pred = mlp_clf.predict(test_text)
        if edit == 1:
          results.append([sizes, d, m, 'stopwords_special', f1_score(test_labels, mlp_pred, average='macro')])
        elif edit == 2:
          results.append([sizes, d, m, 'stemming', f1_score(test_labels, mlp_pred, average='macro')])
        elif edit == 3:
          results.append([sizes, d, m, 'lemmatisation', f1_score(test_labels, mlp_pred, average='macro')])

        if len(results) % 10 == 0:
          print(len(results))

In [ ]:
df_results = pd.DataFrame(results, columns=["hidden_layer_sizes", "Dataset", "Vectorizer", "Preprocess", "F1-score"])
df_results.to_excel("results_mlp1.xlsx", index=False)

In [ ]:
results = []
for sizes in [(50, 60, 70, 80), (80, 100, 100, 100, 100)]:
  for d in ['news', 'imdb']:
    for m in ['count', 'tfidf']:
      for edit in [1, 2, 3]:
        if d == 'news':
          train_texts = train_texts_news
          test_texts = test_texts_news
          train_labels = train_labels_news
          test_labels = test_labels_news
        else:
          train_texts = train_texts_imdb
          test_texts = test_texts_imdb
          train_labels = train_labels_imdb
          test_labels = test_labels_imdb


        train_text, test_text = vectorize(train_texts, test_texts, m, edit)

        mlp_clf = MLPClassifier(hidden_layer_sizes=sizes)
        mlp_clf.fit(train_text, train_labels)
        mlp_pred = mlp_clf.predict(test_text)
        if edit == 1:
          results.append([sizes, d, m, 'stopwords_special', f1_score(test_labels, mlp_pred, average='macro')])
        elif edit == 2:
          results.append([sizes, d, m, 'stemming', f1_score(test_labels, mlp_pred, average='macro')])
        elif edit == 3:
          results.append([sizes, d, m, 'lemmatisation', f1_score(test_labels, mlp_pred, average='macro')])

        if len(results) % 10 == 0:
          print(len(results))

In [ ]:
len(results)

In [ ]:
df_results = pd.DataFrame(results, columns=["hidden_layer_sizes", "Dataset", "Vectorizer", "Preprocess", "F1-score"])
df_results.to_excel("results_mlp2.xlsx", index=False)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

results_lsa = []
for k in [10, 50, 100]:
  for sizes in [(20, 40,), (20, 20, 20,), (10, 10, 20, 20)]:
    for d in ['news', 'imdb']:
      for m in ['count', 'tfidf']:
        for edit in [1, 2, 3]:
          if d == 'news':
            train_texts = train_texts_news
            test_texts = test_texts_news
            train_labels = train_labels_news
            test_labels = test_labels_news
          else:
            train_texts = train_texts_imdb
            test_texts = test_texts_imdb
            train_labels = train_labels_imdb
            test_labels = test_labels_imdb


          train_text, test_text = vectorize(train_texts, test_texts, m, edit)

          svd = TruncatedSVD(n_components=k, random_state=42)
          lsa = make_pipeline(svd)
          train_lsa = lsa.fit_transform(train_text)
          test_lsa = lsa.transform(test_text)

          train_lsa = np.clip(train_lsa, 0, None)
          test_lsa = np.clip(test_lsa, 0, None)

          mlp_clf = MLPClassifier(hidden_layer_sizes=sizes)
          mlp_clf.fit(train_text, train_labels)
          mlp_pred = mlp_clf.predict(test_text)
          if edit == 1:
            results_lsa.append([sizes, d, m, 'stopwords_special', f1_score(test_labels, mlp_pred, average='macro')])
          elif edit == 2:
            results_lsa.append([sizes, d, m, 'stemming', f1_score(test_labels, mlp_pred, average='macro')])
          elif edit == 3:
            results_lsa.append([sizes, d, m, 'lemmatisation', f1_score(test_labels, mlp_pred, average='macro')])

          if len(results_lsa) % 10 == 0:
            print(len(results_lsa))

In [ ]:
len(results_lsa)

In [ ]:
results_lsa

In [ ]:
new_results = []
for i in range(len(results_lsa)):
  if 0 <= i <= 35:
    new_results.append([10] + results_lsa[i])
  elif 36 <= i <= 71:
    new_results.append([50] + results_lsa[i])
  elif 72 <= i <= 107:
    new_results.append([100] + results_lsa[i])

In [ ]:
df_results = pd.DataFrame(new_results, columns=["k", "hidden_layer_sizes", "Dataset", "Vectorizer", "Preprocess", "F1-score"])
df_results.to_excel("results_mlp_lsa.xlsx", index=False)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
results_lsa = []
for k in [10, 50, 100]:
  for sizes in [(20, 20, 30, 50), (50, 60, 70, 80), (80, 100, 100, 100, 100)]:
    for d in ['news', 'imdb']:
      for m in ['count', 'tfidf']:
        for edit in [1, 2, 3]:
          if d == 'news':
            train_texts = train_texts_news
            test_texts = test_texts_news
            train_labels = train_labels_news
            test_labels = test_labels_news
          else:
            train_texts = train_texts_imdb
            test_texts = test_texts_imdb
            train_labels = train_labels_imdb
            test_labels = test_labels_imdb


          train_text, test_text = vectorize(train_texts, test_texts, m, edit)

          svd = TruncatedSVD(n_components=k, random_state=42)
          lsa = make_pipeline(svd)
          train_lsa = lsa.fit_transform(train_text)
          test_lsa = lsa.transform(test_text)

          train_lsa = np.clip(train_lsa, 0, None)
          test_lsa = np.clip(test_lsa, 0, None)

          mlp_clf = MLPClassifier(hidden_layer_sizes=sizes)
          mlp_clf.fit(train_text, train_labels)
          mlp_pred = mlp_clf.predict(test_text)
          if edit == 1:
            results_lsa.append([sizes, d, m, 'stopwords_special', f1_score(test_labels, mlp_pred, average='macro')])
          elif edit == 2:
            results_lsa.append([sizes, d, m, 'stemming', f1_score(test_labels, mlp_pred, average='macro')])
          elif edit == 3:
            results_lsa.append([sizes, d, m, 'lemmatisation', f1_score(test_labels, mlp_pred, average='macro')])

          if len(results_lsa) % 10 == 0:
            print(len(results_lsa))

In [ ]:
new_results = []
for i in range(len(results_lsa)):
  if 0 <= i <= 35:
    new_results.append([10] + results_lsa[i])
  elif 36 <= i <= 71:
    new_results.append([50] + results_lsa[i])
  elif 72 <= i <= 107:
    new_results.append([100] + results_lsa[i])

In [ ]:
df_results = pd.DataFrame(new_results, columns=["k", "hidden_layer_sizes", "Dataset", "Vectorizer", "Preprocess", "F1-score"])
df_results.to_excel("results_mlp_lsa1.xlsx", index=False)